# Test querying classifyer

Example from justin: http://classyfire.wishartlab.com/entities/HNDVDQJCIGZPNO-YFKPBYRVSA-N (histidine)

Fetch the result from above query in json format.

In [24]:
import urllib2
import json
import jsonpickle

def get_json(url):
    response = urllib2.urlopen(url)
    data = json.load(response)   
    return data

Let's see what info we get back from classifyer

In [3]:
url = 'http://classyfire.wishartlab.com/entities/HNDVDQJCIGZPNO-YFKPBYRVSA-N.json'
data = get_json(url)
for key in data:
    print key

kingdom
smiles
inchikey
classification_version
description
predicted_lipidmaps_terms
molecular_framework
alternative_parents
subclass
intermediate_nodes
superclass
substituents
external_descriptors
direct_parent
class
predicted_chebi_terms


We need the kingdom, superclass, class, subclass, intermediate_nodes and direct parent to contruct the taxonomy path of this document (InChiKey). 

Wrap this nicely as a function. We pass in the inchi key and get back the taxonomy.

In [59]:
def get_taxa_path(inchikey):

    url = 'http://classyfire.wishartlab.com/entities/%s.json' % inchikey
    response = urllib2.urlopen(url)
    data = json.load(response)       
    
    # store the taxonomy path for this inchikey here
    taxa_path = []

    # add the top-4 taxa
    keys = ['kingdom', 'superclass', 'class', 'subclass']
    for key in keys:
        if data[key] is not None:
            taxa_path.append(data[key]['name'])

    # add all the intermediate taxa >level 4 but above the direct parent
    for entry in data['intermediate_nodes']:
        taxa_path.append(entry['name'])

    # add the direct parent
    taxa_path.append(data['direct_parent']['name'])

    return taxa_path

inchikey = 'HNDVDQJCIGZPNO-YFKPBYRVSA-N'
tp = get_taxa_path(inchikey)
print '\n'.join(tp)

Chemical entities
Organic compounds
Organic acids and derivatives
Carboxylic acids and derivatives
Amino acids, peptides, and analogues
Amino acids and derivatives
Alpha amino acids and derivatives
Histidine and derivatives


Now try with some Mass2Motif from MassBank. First get all the docs above the default doc-topic threshold (0.05). Retrieve the metadata (inchikey) and pass it to Classifyer.

In [64]:
def print_m2m_taxonomy(m2m_id):
    
    server = 'www.ms2lda.org'
    url = 'http://%s/basicviz/get_parents_metadata/%d' % (server, m2m_id)
    data = get_json(url)

    for metadata_str in data:
        doc = jsonpickle.decode(metadata_str)
        inchikey = doc['InChIKey']
        print doc['annotation'], inchikey
        for taxon in get_taxa_path(inchikey):
            print '-', taxon
        print

### 1. Get the Taxonomy of Documents in the Histidine Mass2Motif (MassBank)

In [65]:
print_m2m_taxonomy(1083)

washington_0873 KRBMQYPTDYSENE-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Organic acids and derivatives
- Carboxylic acids and derivatives
- Amino acids, peptides, and analogues
- Peptides
- Dipeptides

washington_0886 KRBMQYPTDYSENE-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Organic acids and derivatives
- Carboxylic acids and derivatives
- Amino acids, peptides, and analogues
- Peptides
- Dipeptides

washington_0859 KRBMQYPTDYSENE-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Organic acids and derivatives
- Carboxylic acids and derivatives
- Amino acids, peptides, and analogues
- Peptides
- Dipeptides

riken_0373 DOUMFZQKYFQNTF-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Phenylpropanoids and polyketides
- Cinnamic acids and derivatives
- Hydroxycinnamic acids and derivatives
- Coumaric acids and derivatives

riken_0684 CQOVPNPJLQNMDC-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Organic acids and derivatives
- Peptidomimetics
- Hyb

### 2. Get the Taxonomy of Documents in the Adenine Mass2Motif (MassBank)

In [68]:
print_m2m_taxonomy(1367)

washington_0559 UZKQTCBAMSWPJD-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Organoheterocyclic compounds
- Imidazopyrimidines
- Purines and purine derivatives
- 6-aminopurines
- 6-alkylaminopurines

washington_0601 GOSWTRUMMSCNCW-UHFFFAOYSA-N
- Organic compounds
- Nucleosides, nucleotides, and analogues
- Purine nucleosides
- Purine nucleosides

metabolights_0044 UZKQTCBAMSWPJD-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Organoheterocyclic compounds
- Imidazopyrimidines
- Purines and purine derivatives
- 6-aminopurines
- 6-alkylaminopurines

metabolights_0006 LNQVTSROQXJCDD-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Nucleosides, nucleotides, and analogues
- Ribonucleoside 3'-phosphates
- Ribonucleoside 3'-phosphates

ufz_0051 GFFGJBXGBJISGV-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Organoheterocyclic compounds
- Imidazopyrimidines
- Purines and purine derivatives
- 6-aminopurines

ufz_0109 GFFGJBXGBJISGV-UHFFFAOYSA-N
- Chemical entities
- O

### 3. Get the Taxonomy of Documents in the Ferulic Acid Mass2Motif (MassBank)

In [69]:
print_m2m_taxonomy(1430)

washington_0806 IRUHWRSITUYICV-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Phenylpropanoids and polyketides
- Coumarins and derivatives
- Hydroxycoumarins

washington_0627 ARQXEQLMMNGFDU-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Phenylpropanoids and polyketides
- Coumarins and derivatives
- Coumarin glycosides

washington_0222 ZKMLQDNHMSFULN-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Phenylpropanoids and polyketides
- Flavonoids
- Flavones

ipb_0145 ARQXEQLMMNGFDU-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Phenylpropanoids and polyketides
- Coumarins and derivatives
- Coumarin glycosides

riken_0389 HSHNITRMYYLLCV-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Phenylpropanoids and polyketides
- Coumarins and derivatives
- Hydroxycoumarins
- 7-hydroxycoumarins

riken_0404 HSHNITRMYYLLCV-UHFFFAOYSA-N
- Chemical entities
- Organic compounds
- Phenylpropanoids and polyketides
- Coumarins and derivatives
- Hydroxycoumarins
- 7-hydroxyc